In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import bz2

%matplotlib inline

pd.options.display.max_columns = None

In [ ]:
f_in = open("../challenge/bookings.csv.bz2", "rb")
f_out = open("../challenge/bookings.csv", "wb")

f_out.write(bz2.decompress(f_in.read()))

f_out.close()
f_in.close()

In [12]:
f_in = open("../challenge/searches.csv.bz2", "rb")
f_out = open("../challenge/searches.csv", "wb")

f_out.write(bz2.decompress(f_in.read()))

f_out.close()
f_in.close()

# Exercise 1

Count the number of lines in Python for
each file

In [3]:
bz_bookings = bz2.BZ2File("../challenge/bookings.csv.bz2")
line_list_bookings = bz_bookings.readlines()

In [4]:
len(line_list_bookings)

10000011

In [3]:
bz_searches = bz2.BZ2File("../challenge/searches.csv.bz2")

In [4]:
line_list_searches = bz_searches.readlines()

In [5]:
len(line_list_searches)

20390199

# Exercise 2

Top 10 arrival airports in the world in 2013 (using the bookings file)
- To get the total number of passengers for an airport, you can sum the column `pax`, grouping by `arr_port`. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).
- Print the top 10 arrival airports in the standard output, including the number of passengers.
- Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)
- Bonus point: Solve this problem using pandas (instead of any other approach)

### First approximation

We load ten rows to see all the columns.

In [25]:
bookings = pd.read_csv('../challenge/bookings.csv', nrows=10, sep='^', )
bookings.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


Once we know our columns we can load more rows and drop the columns we don't need.

In [8]:
bookings = pd.read_csv('../challenge/bookings.csv', nrows=100000, sep='^', 
                       usecols=['arr_port', 'pax', 'year'])
bookings.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


In [17]:
bookings_2013 = bookings[bookings['year'] == 2013]

top_10_2013 = bookings_2013.groupby('arr_port').sum().sort_values(by='pax', ascending=False).head(10)
top_10_2013.drop('year', axis=1)

,pax
arr_port,
LHR,1006
MCO,861
JFK,795
LAX,761
BKK,747
LAS,732
SFO,705
ORD,686
CDG,676


### Chunks

Let's define a function to process the data.

In [28]:
def data_preprocessing(df, year):
    '''
    Takes a dataframe and a year, filters it by year, drops the year,
    and drops null values.
    Returns a dataframe.
    
    PARAMETERS
    df: Dataframe
    year: Year to filter by
    '''
    df[df['year'] == year]
    df.drop('year', axis=1, inplace=True)
    df.dropna(inplace=True)
    return df

In [29]:
data_iterator = pd.read_csv("../challenge/bookings.csv", chunksize=10**6, 
                           sep='^', usecols=['arr_port', 'pax', 'year'])

chunk_list = []  

# Each chunk is in dataframe format
for data_chunk in data_iterator: 
    data_chunk = data_preprocessing(data_chunk, 2013)
    chunk_list.append(data_chunk)

bookings = pd.concat(chunk_list)

In [30]:
bookings.shape

(10000009, 2)

*Continue a few cells down.*

------------------------------------------------------

In [15]:
bookings.shape

(10000010, 3)

In [21]:
bookings.head(3)

,arr_port,pax,year
0,LHR,-1.0,2013.0
1,CLT,1.0,2013.0
2,CLT,1.0,2013.0


In [16]:
bookings.isna().sum() 

arr_port    0
pax         1
year        1
dtype: int64

In [19]:
bookings.dropna(inplace=True)

In [20]:
bookings.shape

(10000009, 3)

This data treatment is more efficient if we do it in the for loop.

--------------------------------------------------

*Continue here.*

Now that we have a clean dataframe we can perform the same process as in "First approximation". 

Let's aggregate the data by `arr_port` using the `df.pivot_table()` method!!

In [34]:
top_10 = bookings.pivot_table(values='pax', index='arr_port', 
                              aggfunc=np.sum).sort_values(by='pax', ascending=False).head(10)
top_10

,pax
arr_port,
LHR,88809.0
MCO,70930.0
LAX,70530.0
LAS,69630.0
JFK,66270.0
CDG,64490.0
BKK,59460.0
MIA,58150.0
SFO,58000.0


Top 10 airports by total number of passengers 